In [1]:
import os
import string
import sys

import numpy as np
import pandas as pd

In [2]:
%store -r fire_zones
%store -r years
%store -r months

In [3]:
def add_weather_data(df, path, zone):
    data = pd.read_csv(path)
    data = data[["Year", "Month", "Mean Temp (°C)", "Total Precip (mm)"]]
    data[["Zone"]] = zone
    data = data[["Zone", "Year", "Month", "Mean Temp (°C)", "Total Precip (mm)"]]
    return pd.concat([df, data], ignore_index=True)

In [4]:
#code for iterating through directory from geeksforgeeks https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
weather_data = pd.DataFrame()

for zone in fire_zones:
    directory = "data/HistoricalWeatherData/" + zone
    for filename in os.listdir(directory):
        file = os.path.join(directory, filename)
        if os.path.isfile(file):
            weather_data = add_weather_data(weather_data, file, zone)

In [5]:
weather_data.reset_index()
weather_data = weather_data.dropna()
for zone in fire_zones:
    for year in years:
        for month in months:
            weather_df1 = weather_data[weather_data['Zone'] == zone]
            weather_df2 = weather_df1[weather_df1['Year'] == year]
            weather_df3 = weather_df2[weather_df2['Month'] == month]
            weather_data = weather_data.drop(weather_df3.index)
            precip = weather_df3[["Total Precip (mm)"]].sum()[0]
            if (weather_df3.shape[0] > 0):
                avg_temp = weather_df3[["Mean Temp (°C)"]].sum()[0] / weather_df3.shape[0]
            else:
                avg_temp = np.nan
            new_row = pd.DataFrame({'Zone':zone, 'Year':[year], 'Month':[month], 'Mean Temp (°C)':[avg_temp], 'Total Precip (mm)':[precip]})
            weather_data = pd.concat([new_row,weather_data.loc[:]])

In [6]:
weather_data.to_csv("data/weather_data.csv")